# LAMP DPU


In [1]:
from pynq_dpu import DpuOverlay
import numpy as np
import random
import cv2
import colorsys
from PIL import Image
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
%matplotlib inline
from pynq_dpu.edge.dnndk.tf_yolov3_voc_py.tf_yolov3_voc import *

from MPTimeSeriesGenerator import MPTimeseriesGenerator
import scipy.io as sio
import numpy as np
from pynq import Clocks
import threading

'''
Clocks.fclk0_mhz = 50.0
Clocks.fclk1_mhz = 50.0
Clocks.fclk2_mhz = 50.0
Clocks.fclk3_mhz = 50.0

print(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')
print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')
print(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')
print(f'FCLK2: {Clocks.fclk2_mhz:.6f}MHz')
print(f'FCLK3: {Clocks.fclk3_mhz:.6f}MHz')
'''

"\nClocks.fclk0_mhz = 50.0\nClocks.fclk1_mhz = 50.0\nClocks.fclk2_mhz = 50.0\nClocks.fclk3_mhz = 50.0\n\nprint(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')\nprint(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')\nprint(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')\nprint(f'FCLK2: {Clocks.fclk2_mhz:.6f}MHz')\nprint(f'FCLK3: {Clocks.fclk3_mhz:.6f}MHz')\n"

In [2]:
#loads the bit file and the model and creates kernel and task

overlay = DpuOverlay("dpu.bit")
overlay.load_model("dpu_lamp_0.elf")

n2cube.dpuOpen()
kernel = n2cube.dpuLoadKernel("lamp_0")
# uncomment for multithread
#kernel2 = n2cube.dpuLoadKernel("lamp_0")
task = n2cube.dpuCreateTask(kernel, 0)

In [3]:
# prepare the input data

matrix_profile_window = 256
sample_rate = 20
lookbehind_seconds = 0
lookahead_seconds = 0
subsequence_stride = 256
lookbehind = sample_rate * lookbehind_seconds
num_outputs = 256
lookahead = sample_rate * lookahead_seconds
forward_sequences = lookahead + num_outputs
subsequences_per_input = lookbehind + num_outputs + lookahead
channel_stride = 8
n_input_series = 1
subsequences_per_input = subsequences_per_input // channel_stride
high_weight = 1
low_thresh = -1
high_thresh = 1

all_data = sio.loadmat('insect_no_classification.mat')

mp_val = np.array(all_data['mp_val'])
ts_val = np.array(all_data['ts_val'])
#mp = np.zeros((len(ts_val) - matrix_profile_window + 1, 1))

valid_gen = MPTimeseriesGenerator(ts_val, mp_val, num_input_timeseries=1, internal_stride=8, num_outputs=256,lookahead=forward_sequences, lookbehind=lookbehind, important_upper_threshold=high_thresh, important_lower_threshold=low_thresh, important_weight=high_weight, length=256, mp_window=256, stride=num_outputs, batch_size=128)

a = valid_gen[8]

x_test, y_test = a
x_test = np.float32(x_test)

data = x_test[0][np.newaxis,...]


In [5]:
# batch normalization
epsilon=1e-3

N, C, H, W = data.shape
# mini-batch mean
mean = np.mean(data, axis=(0, 2, 3))
# mini-batch variance
variance = np.mean((data - mean.reshape((1, C, 1, 1))) ** 2, axis=(0, 2, 3))
# normalize
X_hat = (data - mean.reshape((1, C, 1, 1))) * 1.0 / np.sqrt(variance.reshape((1, C, 1, 1)) + epsilon)


In [ ]:
#runs the kernel for lamp_0
#TODO: is scale needed
#scale = n2cube.dpuGetInputTensorScale(task, "batch_normalization_1_FusedBatchNormV3_add", 0)

import time

scale_in = n2cube.dpuGetInputTensorScale(task, "conv2d_4_Conv2D", 0)
scale_out = n2cube.dpuGetOutputTensorScale(task, "conv2d_12_Conv2D", 0)

print(scale_in)

feed_data = data #/ scale_in
#print(feed_data)

input_len = n2cube.dpuGetInputTensorSize(task, "conv2d_4_Conv2D")

n2cube.dpuSetInputTensorInHWCFP32(task, "conv2d_1_Conv2D", feed_data, input_len)
n2cube.dpuSetInputTensorInHWCFP32(task, "conv2d_4_Conv2D", feed_data, input_len)

n2cube.dpuRunTask(task)

conv_sbbox_size = n2cube.dpuGetOutputTensorSize(task, "conv2d_12_Conv2D")
conv_out1 = n2cube.dpuGetOutputTensorInHWCFP32(task, "conv2d_12_Conv2D", conv_sbbox_size)

#print(conv_out1)    
conv_out1 = np.reshape(conv_out1, (1, 256, 1, 192))

#for i in range(256):
#    print(conv_out1[0][i][0][2])


In [7]:
out_scaled = conv_out1 / scale_out

global_avg = np.apply_over_axes(np.mean, out_scaled, [1, 2])
print(global_avg)

[[[[  0.00000000e+00   1.03906250e+00   2.34375000e-02   0.00000000e+00
      0.00000000e+00   0.00000000e+00   6.98828125e+00   2.34375000e-02
      0.00000000e+00   1.07070312e+01   5.78515625e+00   1.87539062e+01
      2.05468750e+00   3.25625000e+01   0.00000000e+00   3.28906250e+00
      1.70546875e+01   0.00000000e+00   2.49531250e+01   1.36367188e+01
      3.27695312e+01   7.96093750e+00   0.00000000e+00   3.51562500e-02
      2.10546875e+00   1.90273438e+01   9.83203125e+00   1.56250000e-02
      1.12539062e+01   5.88671875e+00   1.97617188e+01   1.57382812e+01
      0.00000000e+00   0.00000000e+00   3.17929688e+01   0.00000000e+00
      2.49531250e+01   1.86601562e+01   0.00000000e+00   0.00000000e+00
      0.00000000e+00   0.00000000e+00   3.19335938e+01   0.00000000e+00
      0.00000000e+00   2.87500000e+00   0.00000000e+00   2.08632812e+01
      3.17812500e+01   0.00000000e+00   3.90625000e-03   0.00000000e+00
      1.18945312e+01   0.00000000e+00   7.81250000e-02   3.90625

In [8]:
n2cube.dpuDestroyTask(task)
n2cube.dpuDestroyKernel(kernel)


0

In [9]:
# loads the second kernel for dense layer
overlay.load_model("dpu_dense_2.elf")

n2cube.dpuOpen()
kernel = n2cube.dpuLoadKernel("dense_2")
task = n2cube.dpuCreateTask(kernel, 0)


In [10]:
# runs the second kernel for dense multiplicatoin

scale_in = n2cube.dpuGetInputTensorScale(task, "dense_1_MatMul", 0)
scale_out = n2cube.dpuGetOutputTensorScale(task, "dense_1_MatMul", 0)

print(scale_in)
print(scale_out)

feed_input = global_avg / scale_in

input_len = n2cube.dpuGetInputTensorSize(task, "dense_1_MatMul")
n2cube.dpuSetInputTensorInHWCFP32(task, "dense_1_MatMul", feed_input, input_len)

n2cube.dpuRunTask(task)

conv_sbbox_size = n2cube.dpuGetOutputTensorSize(task, "dense_1_MatMul")
conv_out1 = n2cube.dpuGetOutputTensorInHWCFP32(task, "dense_1_MatMul", conv_sbbox_size)

conv_out1 = np.reshape(conv_out1, (1, 1, 1, 256))
print (conv_out1)

64.0
0.03125
[[[[ 0.40625  0.40625  0.375    0.4375   0.375    0.625    0.59375  0.375
     0.46875  0.53125  0.46875  0.75     0.3125   0.40625  0.46875  0.28125
     0.375    0.4375   0.28125  0.34375  0.4375   0.59375  0.40625  0.40625
     0.375    0.5      0.34375  0.625    0.46875  0.25     0.34375  0.5
     0.4375   0.4375   0.375    0.4375   0.5      0.34375  0.46875  0.34375
     0.59375  0.46875  0.53125  0.4375   0.40625  0.59375  0.40625  0.375
     0.5625   0.4375   0.53125  0.53125  0.53125  0.5625   0.46875  0.4375
     0.5      0.5      0.5625   0.46875  0.4375   0.59375  0.5      0.53125
     0.5625   0.46875  0.5      0.46875  0.4375   0.46875  0.5      0.59375
     0.53125  0.625    0.4375   0.53125  0.5625   0.46875  0.46875  0.46875
     0.5625   0.4375   0.59375  0.4375   0.5625   0.46875  0.4375   0.53125
     0.375    0.625    0.53125  0.5      0.4375   0.53125  0.5      0.5
     0.5625   0.4375   0.5625   0.625    0.625    0.46875  0.5625   0.34375
     0.5625 

In [11]:
out_scaled = conv_out1 / scale_out
sigmoid_out = 1/(1 + np.exp(-out_scaled)) 
print(sigmoid_out)

[[[[ 0.99999774  0.99999774  0.9999938   0.99999917  0.9999938   1.          1.
     0.9999938   0.99999964  1.          0.99999964  1.          0.99995458
     0.99999774  0.99999964  0.99987662  0.9999938   0.99999917  0.99987662
     0.99998331  0.99999917  1.          0.99999774  0.99999774  0.9999938
     0.99999988  0.99998331  1.          0.99999964  0.99966466  0.99998331
     0.99999988  0.99999917  0.99999917  0.9999938   0.99999917  0.99999988
     0.99998331  0.99999964  0.99998331  1.          0.99999964  1.
     0.99999917  0.99999774  1.          0.99999774  0.9999938   1.
     0.99999917  1.          1.          1.          1.          0.99999964
     0.99999917  0.99999988  0.99999988  1.          0.99999964  0.99999917
     1.          0.99999988  1.          1.          0.99999964  0.99999988
     0.99999964  0.99999917  0.99999964  0.99999988  1.          1.          1.
     0.99999917  1.          1.          0.99999964  0.99999964  0.99999964
     1.          0.99

In [12]:
y = y_test[0]
y.reshape(1, 256)

sigmoid_out.reshape(1, 256)

print(np.mean(np.abs((sigmoid_out - y) / sigmoid_out)) * 100)


5.03519071538


In [13]:
n2cube.dpuDestroyTask(task)
n2cube.dpuDestroyKernel(kernel)

0

In [94]:
def run_dpu_task(kernel, index):
    task = n2cube.dpuCreateTask(kernel, 0)
    
    scale_in = n2cube.dpuGetInputTensorScale(task, "conv2d_4_Conv2D", 0)
    scale_out = n2cube.dpuGetOutputTensorScale(task, "conv2d_12_Conv2D", 0)
    a = valid_gen[0]

    x_test, y_test = a
    x_test = np.float32(x_test)

    data = x_test[0][np.newaxis,...]
    feed_data = data / scale_in
    for i in range(4):

        

        input_len = n2cube.dpuGetInputTensorSize(task, "conv2d_4_Conv2D")
        n2cube.dpuSetInputTensorInHWCFP32(task, "conv2d_4_Conv2D", feed_data, input_len)
        n2cube.dpuSetInputTensorInHWCFP32(task, "conv2d_1_Conv2D", feed_data, input_len)

        n2cube.dpuRunTask(task)

        conv_sbbox_size = n2cube.dpuGetOutputTensorSize(task, "conv2d_12_Conv2D")
        conv_out1 = n2cube.dpuGetOutputTensorInHWCFP32(task, "conv2d_12_Conv2D", conv_sbbox_size)
    
    n2cube.dpuDestroyTask(task)

threadnum = 8
threadAll = []

for i in range(threadnum):
    if i%2 == 0:
        t1 = threading.Thread(target=run_dpu_task, args=(kernel, i))
        print("1")
    else:
        t1 = threading.Thread(target=run_dpu_task, args=(kernel2, i))
        print("2")
        
    threadAll.append(t1)
for x in threadAll:
    x.start()
for x in threadAll:
    x.join()
    
n2cube.dpuDestroyKernel(kernel)
n2cube.dpuDestroyKernel(kernel)


1


NameError: name 'kernel2' is not defined

In [ ]:
'''
start = time.time()

for i in range(10):

    a = valid_gen[i]

    x_test, y_test = a
    x_test = np.float32(x_test)

    data = x_test[0][np.newaxis,...]
    feed_data = data / scale_in


    input_len = n2cube.dpuGetInputTensorSize(task, "conv2d_4_Conv2D")
    n2cube.dpuSetInputTensorInHWCFP32(task, "conv2d_4_Conv2D", feed_data, input_len)
    n2cube.dpuSetInputTensorInHWCFP32(task, "conv2d_1_Conv2D", feed_data, input_len)

    n2cube.dpuRunTask(task)

    conv_sbbox_size = n2cube.dpuGetOutputTensorSize(task, "conv2d_12_Conv2D")
    conv_out1 = n2cube.dpuGetOutputTensorInHWCFP32(task, "conv2d_12_Conv2D", conv_sbbox_size)

end = time.time()
print(end - start)
'''